In [10]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

# 1、处理数据

## 1-1、下载训练集MNIST 手写数字训练集
torchvision.transforms里面的操作是对导入的图片做处理：
(1)、比如可以随机取(50, 50)这样的窗框大小，或者随机翻转;
(2)、 或者去中间的(50, 50)的窗框大小部分等等;
(3)、但是里面必须要用的是transforms.ToTensor();

In [11]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="./data/",
    train=True,
    download=False,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="./data/",
    train=False,
    download=False,
    transform=ToTensor(),
)

## 1-2、加载数据
① 创建一个 Dataset 对象
② 创建一个 DataLoader 对象
③ 循环这个 DataLoader 对象，将img, label加载到模型中进行训练
    dataset,传入的数据集
    batch_size=1, 每个batch有多少个样本
    shuffle=False,在每个epoch开始的时候，对数据进行重新排序,训练集中True
    sampler=None,自定义从数据集中取样本的策略，如果指定这个参数，那么shuffle必须为False
    batch_sampler=None,与sampler类似，但是一次只返回一个batch的indices（索引）需要注意的是，一旦指定了这个参数，那么batch_size,shuffle,sampler,drop_last就不能再制定了（互斥——Mutually exclusive）
    num_workers=0,这个参数决定了有几个进程来处理data loading。0意味着所有的数据都会被load进主进程。（默认为0）
    collate_fn=None, 将一个list的sample组成一个mini-batch的函数
    pin_memory=False,如果设置为True，那么data loader将会在返回它们之前，将tensors拷贝到CUDA中的固定内存（CUDA pinned memory）中.
    drop_last=False,如果设置为True：这个是对最后的未完成的batch来说的，比如你的batch_size设置为64，而一个epoch只有100个样本，那么训练的时候后面的36个就被扔掉了…
                如果为False（默认），那么会继续正常执行，只是最后的batch_size会小一点。
    timeout=0, 如果是正数，表明等待从worker进程中收集一个batch等待的时间，若超出设定的时间还没有收集到，那就不收集这个内容了。这个numeric应总是大于等于0。默认为0
    worker_init_fn=None,每个worker初始化函数 
    multiprocessing_context=None,

In [12]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size = batch_size)
test_dataloader = DataLoader(test_data, batch_size = batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]", X.shape)
    print("Shape of y:", y.shape, y.dtype)
    break

Shape of X [N, C, H, W] torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


# 2、创建模型
在PyTorch中定义一个神经网络，我们创建了一个从nn.Module继承的类。
我们在__init__函数中定义网络的层，并在函数中指定数据如何通过网络forward。
为了加速神经网络中的操作，我们将其移至GPU（如果有）

In [13]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# Define model
class NeuraNetwork(nn.Module):
    def __init__(self):
        super(NeuraNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuraNetwork().to(device)
print(model)

Using cuda device
NeuraNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


# 3、优化模型参数
要训练模型，我们需要损失函数 和优化器。
定义loss和optimizer
交叉熵损失函数:
    CrossEntropyLoss(),交叉熵主要是用来判定实际的输出与期望的输出的接近程度；
    CrossEntropyLoss()函数的主要是将softmax-log-NLLLoss合并到一块得到的结果

In [15]:
loss_fn = nn.CrossEntropyLoss()
# 随机梯度下降
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3) 

单个训练循环中，模型对训练数据集进行预测（分批进给），然后反向传播预测误差以调整模型的参数。

In [16]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        pred = model(X)
        loss = loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch*len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

对照测试数据集检查模型的性能，以确保模型在学习。

In [20]:
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss:{test_loss:>8f} \n")

迭代训练，模型学习参数去做更好的预测，没次迭代输出模型准确率和loss；
希望看到每次迭代精度提高，loss降低。

In [21]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")

Epoch 1
-------------------
loss: 2.187505 [    0/60000]
loss: 2.158679 [ 6400/60000]
loss: 2.118441 [12800/60000]
loss: 2.131573 [19200/60000]
loss: 2.063815 [25600/60000]
loss: 2.077253 [32000/60000]
loss: 2.083774 [38400/60000]
loss: 2.047268 [44800/60000]
loss: 2.038332 [51200/60000]
loss: 1.975190 [57600/60000]
Test Error: 
 Accuracy: 53.3%, Avg loss:0.031046 

Epoch 2
-------------------
loss: 2.015596 [    0/60000]
loss: 1.949362 [ 6400/60000]
loss: 1.880694 [12800/60000]
loss: 1.911317 [19200/60000]
loss: 1.792747 [25600/60000]
loss: 1.852538 [32000/60000]
loss: 1.852765 [38400/60000]
loss: 1.818456 [44800/60000]
loss: 1.804411 [51200/60000]
loss: 1.701063 [57600/60000]
Test Error: 
 Accuracy: 58.8%, Avg loss:0.026995 

Epoch 3
-------------------
loss: 1.794828 [    0/60000]
loss: 1.690990 [ 6400/60000]
loss: 1.607215 [12800/60000]
loss: 1.643807 [19200/60000]
loss: 1.509737 [25600/60000]
loss: 1.641469 [32000/60000]
loss: 1.611163 [38400/60000]
loss: 1.619523 [44800/60000]
lo

# 4、保存模型
保存模型的常用方法是序列化内部状态字典（包含模型参数）。

In [22]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


# 5、加载模型
加载模型的过程包括重新创建模型结构并将状态字典加载到其中。

In [23]:
model = NeuraNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

现在可以使用该模型进行预测。

In [45]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
